In [48]:
# Import libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
import re
from langchain.embeddings import HuggingFaceHubEmbeddings
from dotenv import load_dotenv
import pandas as pd

In [49]:
# Read file
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Data Warehousing, OLAP, and Data Mining.pdf")
pages = loader.load_and_split()
pages[0]

Document(page_content='Chapter 14. Data Warehousing,OLAP, and Data MiningCHAPTER 14DATA WAREHOUSING, OLAP, AND DATA MININGAfter reading this chapter, the reader will understand:What are OLTP systems?The need of a data warehouseBasic characteristics of a data warehouse, which include subjectoriented, non-volatile, time varying, and integrated characteristicsDifferences between data warehouse and OLTP systemsThe entire process of getting data into the data warehouse, which istermed as extract, transform, and load (ETL) processThe importance of metadata repositoryThe multidimensional data model for a data warehouseThe concept of fact and dimension tablesThe two ways of representing multidimensional data in a datawarehouse, which include star schema and snowflake schemaThe online analytical processing (OLAP) technology, that enablesanalysts, managers, and executives to analyse the complex dataderived from the data warehouseVarious functionalities provided by OLAP system, which includepivot

In [50]:
# Make chunks
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = ' ',
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
docs = text_splitter.split_documents(pages)
len(docs)

86

In [51]:
print(docs[5])

page_content='from multiple sources, stored under a unified schema, at a single site. Itcan successfully answer any ad hoc, complex, statistical or analyticalqueries. The data once gathered can be stored for a longer periodallowing access to historical data. The data warehouses provide the usera single consolidated interface to data, which makes decision-supportqueries easier to write.Learn MoreA data mart is a subset of data warehouse that is designed for aparticular department of an organization such as sales, marketing, orfinance.NOTE A query that is run at the spur of the moment, and generally isnever saved to run again is known as ad hoc query. These queries aregenerally not predefined, and are built and run as and when required.Since data warehouses have been developed in numerous organizationsto meet their specific needs, there is no single and standard definition ofthis term. According to William Inmon, the father of the modern datawarehouse, a data warehouse is a subject-orien

In [52]:
# Embed text and store in vector space
from langchain.embeddings import HuggingFaceHubEmbeddings
load_dotenv()
# repo_id = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
repo_id = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceHubEmbeddings(
    repo_id= repo_id
)

# load it into Chroma db
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, hf)


c:\Users\harsh\Desktop\NEU\LLM Project\.venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [53]:
# Query on similarity search
query = "What is a Data Warehouse?"
result = db.similarity_search(query, k = 10)

In [54]:
output = result[0].page_content
print(output)

warehouse, a data warehouse is a subject-oriented, integrated,time-variant, non-volatile collection of data in support ofmanagement’s decisions.6. The main advantage of using a data warehouse is that a data analystcan perform complex queries and analyses on the information storedin data warehouse without affecting the OLTP systems.7. A data warehouse basically consists of three components, namely,the data sources, the ETL, and the schema of data stored in datawarehouse including the metadata.8. While creating a data warehouse, the data is extracted from multiple,heterogeneous data sources. Then data cleansing is performed toremove any kind of minor errors and inconsistencies. This data isthen transformed to accommodate semantic mismatches. Thecleaned and transformed data is finally loaded into the warehouse.9. The entire process of getting data into the data warehouse is calledextract, transform, and load (ETL) process.10. The third and the most important component of the data


In [55]:
# Minimum scores is the best score
similarity_scores = {i:d[1] for i,d in enumerate(db.similarity_search_with_score(query, k=5 ))}

ss = pd.Series(similarity_scores)

best_idx = ss.idxmin()

output_based_on_scores = result[best_idx].page_content
print(output_based_on_scores)

warehouse, a data warehouse is a subject-oriented, integrated,time-variant, non-volatile collection of data in support ofmanagement’s decisions.6. The main advantage of using a data warehouse is that a data analystcan perform complex queries and analyses on the information storedin data warehouse without affecting the OLTP systems.7. A data warehouse basically consists of three components, namely,the data sources, the ETL, and the schema of data stored in datawarehouse including the metadata.8. While creating a data warehouse, the data is extracted from multiple,heterogeneous data sources. Then data cleansing is performed toremove any kind of minor errors and inconsistencies. This data isthen transformed to accommodate semantic mismatches. Thecleaned and transformed data is finally loaded into the warehouse.9. The entire process of getting data into the data warehouse is calledextract, transform, and load (ETL) process.10. The third and the most important component of the data


In [11]:
# text = ""
# pdf_text = PdfReader("Data Warehousing, OLAP, and Data Mining.pdf")
# for page_num in range(len(pdf_text.pages)):
#     page_text = pdf_text.pages[page_num]
#     text += page_text.extract_text()


# text = re.sub(' +', ' ', text)
# print(text)


In [20]:
# text_splitter = CharacterTextSplitter(        
#     separator = "\n",
#     chunk_size = 1000,
#     chunk_overlap  = 200,
#     length_function = len,
#     is_separator_regex = False,
# )

# chunks_text = text_splitter.split_text(text)
# chunks_text

In [25]:
# repo_id = "sentence-transformers/all-mpnet-base-v2"
# hf = HuggingFaceHubEmbeddings(
#     repo_id= repo_id
# )

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# embeddings = model.encode(chunks_text)
# print(embeddings)

In [26]:
# load it into Chroma db
# from langchain.vectorstores import Chroma
# db = Chroma.from_texts(text, hf)

In [27]:
# Chroma.from_texts(text, hf)